<a href="https://colab.research.google.com/github/kethlyncampos/webscraper/blob/main/web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install requests
!pip install iso8601

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [54]:
import requests
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
import pytz
from datetime import datetime
import iso8601
from dateutil import tz
from google.colab import drive
from selenium.webdriver.common.by import By
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [10]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [55]:
yahoo = pd.read_csv('/content/gdrive/MyDrive/news_yahoo.csv')
infomoney = pd.read_csv('/content/gdrive/MyDrive/news_infomoney.csv', lineterminator='\n')

In [56]:
comp_yahoo = []
comp_infomoney = []

for c in yahoo['URL']:
  comp_yahoo.append(c)

for c in infomoney['URL']:
  comp_infomoney.append(c)

In [80]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-notifications")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36")
browser = webdriver.Chrome(options=options)

In [75]:
def req_yahoo(url):
  browser.get(url)
  scrollar()
  links = browser.find_elements(By.CSS_SELECTOR, 'a.js-content-viewer.rapidnofollow')
  noticias = []

  for link in links:
    if (('https://br.yahoo.com/news/' not in link.get_attribute('href')) and (link.get_attribute('href') not in comp_yahoo)):
      browser.execute_script("arguments[0].click();", link)
      scrollar()
      WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
      temp = BeautifulSoup(browser.page_source, 'html.parser')
      title = temp.find('h1', attrs={'data-test-locator':'headline'}).text
      author = temp.find('span', attrs={'class':'caas-author-byline-collapse'}).text
      collect_date = convert_timezone(datetime.now())
      publication_date = convert_timezone(iso8601.parse_date(temp.find('time')['datetime']))
      description = temp.find('div', attrs={'class':'caas-body'})
      try:
        description.figure.decompose()
        description = description.text
      except:
        description = description.text
      noticias.append(['Yahoo Finanças',title, description, author, publication_date, collect_date, browser.current_url])
      browser.back()
      sleep(5)

  return noticias

In [59]:
def req_infomoney(url):
  browser.get(url)
  browser.implicitly_wait(10)
  browser.maximize_window()
  alert = browser.switch_to.active_element.click
  infinite = browser.find_element(By.ID, 'infinite-handle')
  browser.execute_script("arguments[0].click();", infinite)
  count = 40
  while(count > 0):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(5)
    browser.execute_script("arguments[0].click();", infinite)
    newHeight = browser.execute_script("return document.body.scrollHeight")
    lastHeight = newHeight
    count-=1
  return BeautifulSoup(browser.page_source, 'html.parser')

In [60]:
def scrollar():
  lastHeight = browser.execute_script("return document.body.scrollHeight")
  while True:
      browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      sleep(5)
      newHeight = browser.execute_script("return document.body.scrollHeight")
      if newHeight == lastHeight:
          break
      lastHeight = newHeight
  return

In [61]:
def convert_timezone(date_time):
  sp = tz.gettz('America/Sao_Paulo')
  new_date = date_time.replace(tzinfo=pytz.utc)
  new_date = new_date.astimezone(sp)
  new_date = new_date.strftime('%Y-%m-%d %H:%M:%S')
  return new_date

In [62]:
def trata_string(data):
  data = re.sub(r'[\t\n ]+', ' ', data)
  data = re.sub(r'(?s)<figure.*?</figure>', ' ', data)
  return data

In [81]:
array_noticias = req_yahoo('https://br.yahoo.com/topics/bolsa-de-valores/')

https://br.financas.yahoo.com/noticias/2-supermercado-responde-por-80-154600120.html
entrei
https://br.financas.yahoo.com/noticias/mercado-de-cannabis-conheca-5-empresas-que-movimentam-milhoes-080021118.html
entrei
https://br.financas.yahoo.com/noticias/entenda-o-que-%C3%A9-greenwashing-164700173.html
entrei
https://br.financas.yahoo.com/noticias/investidor-da-bolsa-ainda-mira-160700421.html
entrei
https://br.financas.yahoo.com/noticias/queda-mercado-e-novidades-da-132316386.html
entrei
https://br.yahoo.com/finance/news/press%C3%A3o-internacional-por-preserva%C3%A7%C3%A3o-que-201954902.html
https://br.yahoo.com/finance/news/panela-el%C3%A9trica-japonesa-torna-frango-155806972.html
https://br.yahoo.com/finance/news/infla%C3%A7%C3%A3o-da-carne-abre-espa%C3%A7o-204519379.html
https://br.yahoo.com/finance/news/presidente-m%C3%A9xico-escolhe-subsecret%C3%A1ria-fazenda-143253998.html
entrei
https://br.noticias.yahoo.com/produtores-soja-tacham-protecionista-plano-202825625.html
entrei
https:/

In [ ]:
array_noticias

In [82]:
dataset_temp = pd.DataFrame(array_noticias,columns=['Fonte', 'Título', 'Descrição', 'Autor', 'Data/Hora de Publicação', 'Data/Hora de Coleta', 'URL'])

In [83]:
dataset_temp.drop_duplicates(subset=['Descrição'], inplace = True)

In [84]:
dataset_temp.reset_index(drop = True, inplace = True)

In [85]:
dataset_temp

,Fonte,Título,Descrição,Autor,Data/Hora de Publicação,Data/Hora de Coleta,URL
0,Yahoo Finanças,2 - Supermercado responde por 80% dos mais ven...,"SÃO PAULO, SP (FOLHAPRESS) - Dos dez produtos ...",PAULA SOPRANA,2021-11-28 12:46:00,2021-11-28 14:52:45,https://br.yahoo.com/noticias/2-supermercado-r...
1,Yahoo Finanças,Mercado de cannabis: conheça 5 empresas que mo...,"Com o avanço da regulamentação da maconha, sej...",Redação Finanças,2021-11-28 05:00:21,2021-11-28 14:53:01,https://br.yahoo.com/noticias/investidor-da-bo...
2,Yahoo Finanças,"Entenda o que é greenwashing, o blá-blá-blá ve...","BELO HORIZONTE, MG (FOLHAPRESS) - Uma empresa ...",THIAGO BETHÔNICO,2021-11-27 13:47:00,2021-11-28 14:53:17,https://br.yahoo.com/noticias/queda-mercado-e-...
3,Yahoo Finanças,Investidor da Bolsa ainda mira curto prazo e i...,"BELO HORIZONTE, MG (FOLHAPRESS) - A ideia por ...",THIAGO BETHÔNICO,2021-11-27 13:07:00,2021-11-28 14:53:33,https://br.yahoo.com/noticias/entenda-o-que-%C...
4,Yahoo Finanças,Queda do mercado e novidades da Cardano são de...,BeInCrypto –O Bitcoin (BTC) teve mais uma sema...,Anderson Mendes,2021-11-27 10:23:16,2021-11-28 14:53:49,https://br.yahoo.com/noticias/entenda-o-que-%C...
5,Yahoo Finanças,Presidente do México escolhe subsecretária de ...,CIDADE DO MÉXICO (Reuters) - O presidente do M...,,2021-11-24 11:32:53,2021-11-28 14:54:05,https://br.yahoo.com/finance/news/infla%C3%A7%...
6,Yahoo Finanças,Produtores de soja tacham de 'protecionista' p...,A proposta da União Europeia (UE) de proibir a...,,2021-11-23 17:28:25,2021-11-28 14:54:21,https://br.yahoo.com/finance/news/infla%C3%A7%...
7,Yahoo Finanças,TV Brasil faz correção de questões do segundo ...,Os veículos da Empresa Brasil de Comunicação (...,Agência Brasil,2021-11-28 07:39:00,2021-11-28 14:54:37,https://br.yahoo.com/tv-brasil-faz-corre%C3%A7...
8,Yahoo Finanças,"Eleição quebra uma barreira, diz 1ª mulher que...","MOGI DAS CRUZES, SP, E SÃO PAULO, SP (FOLHAPRE...",GÉSSICA BRANDINO E RENATA GALF,2021-11-27 07:30:00,2021-11-28 14:54:53,https://br.yahoo.com/elei%C3%A7%C3%A3o-quebra-...
9,Yahoo Finanças,México exigirá visto de brasileiros a partir d...,CIDADE DO MÉXICO (Reuters) - O México anunciou...,,2021-11-26 20:34:04,2021-11-28 14:55:09,https://br.yahoo.com/patricia-vanzolini-%C3%A9...


In [90]:
yahoo = pd.concat([dataset_temp,yahoo],ignore_index=True)

In [92]:
yahoo.to_csv('/content/gdrive/MyDrive/news_yahoo.csv')

In [53]:
page = req_infomoney('https://www.infomoney.com.br/mercados/')

In [34]:
links = page.find_all(class_="hl-title")

In [37]:
lista_url = []
for j in links:
  aux = j.find('a')
  if(aux):
    if "stock-pickers" not in aux['href']:
      lista_url.append(aux['href'])
lista_url = set(lista_url)

In [94]:
noticias = []
subtitle = ""
for k in lista_url:
  if k not in comp_infomoney:
    response = requests.get(k)
    temp = BeautifulSoup(response.content, 'html.parser')
    title = temp.find('h1', attrs={'class': 'page-title-1'})
    if title: # se não houver título, a página é do tipo cotações e possui apenas as informações sobre a empresa
      title = title.text
      subtitle = trata_string(temp.find('p', attrs={'class': 'article-lead'}).text)
      author = temp.find('span', attrs={'class':'author-name'})
      if author:
        author = trata_string(author.find('a').text) # se não houver autor, a página é do tipo guias
        collect_date = convert_timezone(datetime.now())
        publication_date = iso8601.parse_date(temp.find('time', attrs={'class':'entry-date'})['datetime']).strftime('%Y-%m-%d %H:%M:%S')
        description = temp.find('div', attrs={'class':'article-content'})
        try:
          description.figure.decompose()
          description = description.text
        except:
          description = description.text
        description = trata_string(description)
        noticias.append(['Infomoney',title, subtitle, description, author, publication_date, collect_date, k])

In [44]:
dataset_temp = pd.DataFrame(noticias,columns=['Fonte', 'Título', 'Subtítulo','Descrição', 'Autor', 'Data/Hora de Publicação', 'Data/Hora de Coleta', 'URL'])

In [46]:
dataset_temp.drop_duplicates(subset=['Descrição'], inplace = True)

In [47]:
dataset_temp.reset_index(drop = True, inplace = True)

In [48]:
dataset_temp

,Fonte,Título,Subtítulo,Descrição,Autor,Data/Hora de Publicação,Data/Hora de Coleta,URL
0,Infomoney,Vera Brandimarte é nomeada presidente do Conse...,Jornalista assume a função com o objetivo de a...,SÃO PAULO – A XP anuncia hoje a contratação d...,Equipe InfoMoney,2021-11-10 16:33:09,2021-11-28 13:57:19,https://www.infomoney.com.br/mercados/vera-bra...
1,Infomoney,"China pede plano para que Didi, dona da 99, se...",Órgão fiscalizador de tecnologia da China quer...,Reguladores chineses pediram aos principais e...,Bloomberg,2021-11-26 13:51:25,2021-11-28 13:57:19,https://www.infomoney.com.br/mercados/china-pe...
2,Infomoney,Banco Pine (PINE4) reverte prejuízo e registra...,A receita de prestação de serviços e tarifas s...,O Banco Pine (PINE4) reportou lucro líquido d...,Equipe InfoMoney,2021-11-11 06:47:19,2021-11-28 13:57:19,https://www.infomoney.com.br/mercados/banco-pi...
3,Infomoney,Goldman prevê que ciclo de aperto do Fed será ...,"Segundo economistas, banco central dos EUA dev...",(Bloomberg) – Economistas do Goldman Sachs ac...,Bloomberg,2021-11-25 11:36:29,2021-11-28 13:57:19,https://www.infomoney.com.br/mercados/goldman-...
4,Infomoney,Bolsonaro diz que está buscando rever política...,Com a alta dos combustíveis comprometendo sua ...,"Em mais um ataque à Petrobras (PETR3; PETR4),...",Estadão Conteúdo,2021-11-24 07:49:00,2021-11-28 13:57:20,https://www.infomoney.com.br/politica/bolsonar...
...,...,...,...,...,...,...,...,...
362,Infomoney,"Wiz (WIZS3) registra queda de 72,7% no lucro n...",O resultado reflete efeitos do término do perí...,SÃO PAULO – A Wiz (WIZS3) reportou lucro líqu...,Equipe InfoMoney,2021-11-12 11:15:42,2021-11-28 13:59:37,https://www.infomoney.com.br/mercados/wiz-wizs...
363,Infomoney,"Ação da Nvidia salta 8,25% na Nasdaq com resul...","As ações chegaram a subir 12% na sessão, levan...",SÃO PAULO – As ações da fabricante de chips N...,Equipe InfoMoney,2021-11-18 18:55:21,2021-11-28 13:59:37,https://www.infomoney.com.br/mercados/acao-da-...
364,Infomoney,Bradesco BBI coloca Petrobras (PETR4) como açã...,"Analistas trocaram preferências de PetroRio, P...",SÃO PAULO – O Bradesco BBI reorganizou as sua...,Equipe InfoMoney,2021-11-19 10:55:10,2021-11-28 13:59:37,https://www.infomoney.com.br/mercados/bradesco...
365,Infomoney,"IRB (IRBR3) tem queda de 27,8% no prejuízo do ...",Resultado líquido contábil foi negativamente i...,SÃO PAULO – O IRB Brasil (IRBR3) registrou qu...,Equipe InfoMoney,2021-11-11 20:48:36,2021-11-28 13:59:38,https://www.infomoney.com.br/mercados/irb-irbr...


In [ ]:
infomoney = pd.concat([dataset_temp, infomoney])

In [ ]:
infomoney

In [50]:
dataset_temp.to_csv('/content/gdrive/MyDrive/news_infomoney.csv')